# ENABLING NTP SERVER 

<div class="alert alert-block alert-info">
<i>Hello User.!<br></i> Welcome to <b>python-iLOrest-library</b> <br> This is the example to <b>enable ntp servers .</b> <br> As you are here let's get it done easily.<br>
    To perform this u need to have <a href = "https://github.com/HewlettPackard/python-ilorest-library" style = "">python-ilorest-library</a> installed in your machine.<br>
    And please keep your username and password handy!
    <br>
    <b>Also do execute each successive cell once to avoid any errors!</b>
    
</div>


<div class = "alert alert-block alert-success">
<h1>1</h1>    
In the beginning, we are having some import statements that are actually providing us with the required modules and libraries.
for proper execution of our code.</div>

In [ ]:

import sys
import json
import getpass
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError


<div class = "alert alert-block alert-success">
    <h1>2</h1>   
Below function describes the process how to enable NTP servers for HPE ilo systems. NTP or Network Time Protocol is an internet protocol used to synchronize with computer clokc time sources in a network. 
Current cell is the explains the processes performed inside the below function.
    <br>
    <i>Let me give you a walk through of it!</i>
    <br>
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to obtain the power metrics information.

1. Initially, to obtain NTP server related information for HPE ilo systems a series of HTTP GET requests are being made so as to navigate to that path inside the iLO rest api where required information is stored.
After completion of these get requests we will be having the relevant uri in <b>manager_ethernet_interfaces_members</b>.
<br>
2. Else if <b>resource directory</b> is enabled we can just use the resource directory to and then again navigate through the api to find relevant uri.
<br>
    
3. Whatever changes need to be made to enable the NTP server are being enclosed inside the dictionary named as <b>body</b>. Next up for all the instances of ethernet in ethernet_data we simply make a <b>HTTP PATCH</b> request by passing the ethernet instance and body dictionary with the PATCH request. 
    

4. Following the patch request, the response returned by server is being saved inside resp variable and later if the server returns proper response it means there is no error else error is returned with error information. A reference for json.dumps
function as it is used multiple times - <a href="https://www.geeksforgeeks.org/json-dumps-in-python/">click here</a>.



 </div>

In [7]:
def enable_ntp(_redfishobj, ntp_servers, DISABLE_RESOURCE_DIR):

    ethernet_data = {}

    resource_instances = redfishobj.get_resource_directory()
    if DISABLE_RESOURCE_DIR or not resource_instances:
        #if we do not have a resource directory or want to force it's non use to find the
        #relevant URI
        managers_uri = _redfishobj.root.obj['Managers']['@odata.id']
        managers_response = _redfishobj.get(managers_uri)
        managers_members_uri = next(iter(managers_response.obj['Members']))['@odata.id']
        managers_members_response = _redfishobj.get(managers_members_uri)
        manager_ethernet_interfaces = managers_members_response.obj['EthernetInterfaces']\
                                                                                    ['@odata.id']
        manager_ethernet_interfaces_response = _redfishobj.get(manager_ethernet_interfaces)
        manager_ethernet_interfaces_members = manager_ethernet_interfaces_response.\
                                                            obj['Members']
        for _member in manager_ethernet_interfaces_members:
            _tmp = _redfishobj.get(_member['@odata.id']).obj
            ethernet_data[_member['@odata.id']] = _tmp
    else:
        #Use Resource directory to find the relevant URI
        for instance in resource_instances:
            if '#EthernetInterfaceCollection.' in instance['@odata.type'] and 'Managers' in \
                                                                        instance['@odata.id']:
                ethernet_uri = instance['@odata.id']
                ethernet_interfaces = _redfishobj.get(ethernet_uri).obj['Members']
                for _ethernet_interface in ethernet_interfaces:
                    ethernet_data[_ethernet_interface['@odata.id']] = _redfishobj.\
                                                        get(_ethernet_interface['@odata.id']).dict
                break

    if ethernet_data:
        print("\n\nShowing all available ethernet management interfaces before changes:\n\n")
        print(json.dumps(ethernet_data, indent=4, sort_keys=True))

    body = {"Oem": {"Hpe": {"DHCPv4": {"UseNTPServers": ntp_servers}, \
                            "DHCPv6": {"UseNTPServers": ntp_servers}}}}
    for ethernet in ethernet_data:
        resp = _redfishobj.patch(ethernet, body)
        if resp.status == 400:
            try:
                print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, \
                                                                                sort_keys=True))
            except Exception as excp:
                sys.stderr.write("A response error occurred, unable to access iLO Extended "\
                                 "Message Info...")
        elif resp.status != 200:
            sys.stderr.write("An http response of \'%s\' was returned.\n" % resp.status)
        else:
            print("Success!\n")
            print(json.dumps(resp.dict, indent=4, sort_keys=True))
            _data = _redfishobj.get(ethernet).dict
            sys.stdout.write("\nShowing \'%s\' interface after changes:\n" % ethernet)
            print(json.dumps(_data, indent=4, sort_keys=True))


<div class="alert alert-block alert-success">
<h1>3</h1>   
<b>The Main function: </b> PLEASE ENTER YOU LOGIN CREDENTIALS HERE TO PROCEED FURTHER<br>
<ul>
        
<li>Once your credentials that are enter everything a redfish object is created with the help of RedfishClient class and a session is created and now you will be logged in to the server.</li>
<li>If server is down or Unreachable we get server not reachable message!</li>
<li>If not then we make a call to <b>enable_ntp function</b> by passing parameters such as redfishclient object and enable flag to perform required operation.</li>
<li>After getting required information <i><b>Redfish.logout()</b></i> closes the particular session and logs us out of the system.</li>
    </ul>
</div>

In [ ]:
if __name__ == "__main__":
    
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Name: ")
    LOGIN_PASSWORD = getpass.getpass("Enter password: ")

    # Properties
    # NTP Servers Enable/Disable Flag
    NTP_SERVERS = True
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, \
                                                                            password=LOGIN_PASSWORD)
        # Login with the Redfish client
        REDFISHOBJ.login()
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()

    enable_ntp(REDFISHOBJ, NTP_SERVERS, DISABLE_RESOURCE_DIR)
    REDFISHOBJ.logout()


<div class = "alert alert-block alert-warning">
    <b>When running on the server locally use the following commented values</b><br>
 SYSTEM_URL = None<br>
 LOGIN_ACCOUNT = None<br>
 LOGIN_PASSWORD = None<br>

   <b>When running remotely</b> connect using the secured (https://) address,
     account name, and password to send https requests
    <br>
     SYSTEM_URL acceptable examples:<br>
     https://10.0.0.100<br>
     https://ilo.hostname
</div>

<div class = "alert alert-block alert-success">In case you need help 🔗 links to relevant resources :
    <br>
    1.Python-ilorest-library:<a href = "https://github.com/HewlettPackard/python-ilorest-library">LINK</a><br>
    2.Hpe ilorest-api explorer:<a href = "https://ilorestfulapiexplorer.ext.hpe.com/">LINK</a><br>
    3.Rest api: <a href = "https://restfulapi.net/">LINK</a><br>
    
</div>